In [1]:
import pandas as pd
import os
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV

from sklearn.pipeline import Pipeline
import joblib
from sklearn import linear_model
import pickle

In [2]:
data=pd.read_csv('C:\\Users\\shaun\\OneDrive\\Desktop\\My_Files\\Codes\\Self Project 2\\Data_Set\\corona_fake.csv')

In [3]:
data

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake
...,...,...,...,...
1159,Could the Power of the Sun Slow the Coronavirus?,A study suggests that ultraviolet rays could s...,https://www.nytimes.com/,TRUE
1160,Key evidence for coronavirus spread is flawed ...,"Last week, a medical journal reported that a b...",https://www.nytimes.com/,TRUE
1161,Summer Heat May Not Diminish Coronavirus Strength,"A new report, sent to the White House science ...",https://www.nytimes.com/,TRUE
1162,How Long Will a Vaccine Really Take?,A vaccine would be the ultimate weapon against...,https://www.nytimes.com/,TRUE


In [4]:
data['label'].value_counts()

TRUE    584
Fake    345
fake    230
Name: label, dtype: int64

### Replacing Labels

In [5]:
data["label"]= data["label"].str.replace("fake", "FAKE", case = False) 

In [6]:
data["label"]= data["label"].str.replace("Fake", "FAKE", case = False) 

In [7]:
data['label'].value_counts()

TRUE    584
FAKE    575
Name: label, dtype: int64

### Filling Rows with Missing Labels

In [8]:
label_column=data['label'].isnull()
label_column

for i in range (1164):
    if(label_column[i]==True):
        print(i)
########################################
data.loc[5]['label'] = 'FAKE'
data.loc[15]['label'] = 'TRUE'
data.loc[43]['label'] = 'FAKE'
data.loc[131]['label'] = 'TRUE'
data.loc[242]['label'] = 'FAKE'

5
15
43
131
242


# Testing Fake News Normal Algorithm

In [9]:
data_trial=data
data_trial.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,FAKE
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,FAKE
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,FAKE
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,FAKE
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,FAKE


In [10]:
data_trial.isnull().sum()

title     82
text      10
source    20
label      0
dtype: int64

In [11]:
data_trial=data_trial.fillna(' ')
print(data_trial.isnull().sum())

title     0
text      0
source    0
label     0
dtype: int64


In [12]:
data_trial['total']=data_trial['text']+' '+data_trial['title']

In [13]:
data_trial.head(10)

,title,text,source,label,total
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,FAKE,"You just need to add water, and the drugs and ..."
1,,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,FAKE,Hydroxychloroquine has been shown to have a 10...
2,,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,FAKE,Fact: Hydroxychloroquine has been shown to hav...
3,,The Corona virus is a man made virus created i...,JoanneWrightForCongress,FAKE,The Corona virus is a man made virus created i...
4,,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,FAKE,Doesn’t @BillGates finance research at the Wuh...
5,CORONA UNMASKED: Chinese Intelligence Officer ...,,,FAKE,CORONA UNMASKED: Chinese Intelligence Office...
6,,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,FAKE,Urgent: Health Bulletin to the Public. Ministr...
7,,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,FAKE,"Pls tell ur families, relatives and friendsMOH..."
8,,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,FAKE,SERIOUS EXCELLENT ADVICE by Japanese doctors t...
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE,Stay aware of the latest information on the CO...


In [14]:
#trial=data_trial['total'][0]
#trial=re.sub('[^\w\s]','',trial)
#trial=trial.lower()
#trial

In [15]:
#pred=tfidf_vectorizer.transform(trial) 

In [16]:
data_trial['total'] = data_trial['total'].str.replace('[^\w\s]','')
data_trial['total'] = data_trial['total'].str.lower()

In [17]:
data_trial['total'][0]

'you just need to add water and the drugs and vaccines are ready to be administered there are two parts to the kit one holds pellets containing the chemical machinery that synthesises the end product and the other holds pellets containing instructions that telll the drug which compound to create mix two parts together in a chosen combination add water and the treatment is ready due to the recent outbreak for the coronavirus covid19 the world health organization is giving away vaccine kits just pay 495 for shipping'

In [18]:
data_trial['total'][45]

'the covid2019 epidemic which began in the chinese city of wuhan managed to get out of china and spread throughout the world in two months in midmarch an article became popular in social networks according to which a dangerous strain was artificially created as part of a study of bat coronavirus by american scientists the scientific work was written by a team of biologists back in 2015 on november 9 of the same year it was published on the scientific portal nature  researchers studied the shco14 virus found in chinese horseshoe bats scientists combined this strain with another  its name is sars and it causes a severe acute respiratory syndrome or sars further the researchers generated and characterized a chimeric virus that can  multiply in the primary cells of the human respiratory tract in other words biologists altered the coronavirus of bats so much that they got a strain that could infect a person  and the virus would cause severe pneumonia in him among the authors of the article 

## Train Test Split

In [19]:
y=data_trial.label
y

0       FAKE
1       FAKE
2       FAKE
3       FAKE
4       FAKE
        ... 
1159    TRUE
1160    TRUE
1161    TRUE
1162    TRUE
1163    TRUE
Name: label, Length: 1164, dtype: object

In [20]:
data_trial.drop("label", axis=1,inplace=True)
data_trial.head(10) 

,title,text,source,total
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,you just need to add water and the drugs and v...
1,,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,hydroxychloroquine has been shown to have a 10...
2,,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,fact hydroxychloroquine has been shown to have...
3,,The Corona virus is a man made virus created i...,JoanneWrightForCongress,the corona virus is a man made virus created i...
4,,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,doesnt billgates finance research at the wuhan...
5,CORONA UNMASKED: Chinese Intelligence Officer ...,,,corona unmasked chinese intelligence officer...
6,,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,urgent health bulletin to the public ministry ...
7,,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,pls tell ur families relatives and friendsmoh ...
8,,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,serious excellent advice by japanese doctors t...
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,stay aware of the latest information on the co...


In [21]:
X_train, X_test, y_train, y_test = train_test_split(data_trial['total'], y, test_size=0.2,random_state=102)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(931,) (931,)
(233,) (233,)


In [22]:
X_train.head()

882     viruses cannot travel on radio wavesmobile net...
1053    population reduction is among the goals of the...
404     so bill gates is launching implantable chips t...
502     this video shows why big telecom loves coronav...
68      the coronavirus is a highly infectious disease...
Name: total, dtype: object

## Cleaning Data using NLP Techniques

In [23]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.80)  
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 
tfidf_test = tfidf_vectorizer.transform(X_test)

## Classification Algorithms

### Logistic Regression

In [25]:
logreg = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1, verbose=3, max_iter=300).fit(tfidf_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.5min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


In [26]:
pred = logreg.predict(tfidf_test)
print('Accuracy of Logistic classifier on test set:',accuracy_score(y_test, pred))

Accuracy of Logistic classifier on test set: 0.9399141630901288


In [27]:
results1=confusion_matrix(y_test, pred)
print(results1)

[[118   8]
 [  6 101]]


In [28]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

        FAKE       0.95      0.94      0.94       126
        TRUE       0.93      0.94      0.94       107

    accuracy                           0.94       233
   macro avg       0.94      0.94      0.94       233
weighted avg       0.94      0.94      0.94       233



In [29]:
precision0 = (results1[0][0]/(results1[0][1]+results1[0][0]))
print('Precision=',precision0)

recall0 = (results1[0][0]/(results1[1][0]+results1[0][0]))
print('Recall=',recall0)

Precision= 0.9365079365079365
Recall= 0.9516129032258065


In [45]:
coefficients=logreg.coef_
print(coefficients)

[[ 4.30793651e+00 -2.03698191e-02  8.81305544e-04 ... -2.07244821e-01
  -1.04643500e-01 -8.92310827e-01]]


### Saving Model via Pickle

In [77]:
logreg_model = open('logreg_model.sav', 'wb')
pickle.dump(logreg, logreg_model)
logreg_model.close()

In [78]:
filename = 'logreg_model.sav'
saved_model = pickle.load(open(filename, 'rb'))

In [79]:
saved_model.score(tfidf_test,y_test)

0.9399141630901288

### Random Classifier

In [54]:
rfc = RandomForestClassifier()


In [55]:
rfc.fit(tfidf_train, y_train)
pred2 = rfc.predict(tfidf_test)

Accuracy of Random Forest classifier on test set: 0.93


In [62]:
print('Accuracy of Random Forest classifier on test set: ',accuracy_score(y_test,pred2))

Accuracy of Random Forest classifier on test set:  0.927038626609442


In [56]:
results2=confusion_matrix(y_test, pred2)
print(results2)

[[116  10]
 [  7 100]]


In [57]:
precision2 = (results2[0][0]/(results2[0][1]+results2[0][0]))
print('Prcision=',precision2)

recall2 = (results2[0][0]/(results2[1][0]+results2[0][0]))
print('Recall=',recall2)

Prcision= 0.9206349206349206
Recall= 0.943089430894309


In [58]:
f1_score2=((2*precision2*recall2)/precision2+recall2)
print('F1 score=',f1_score2)

F1 score= 2.8292682926829267


### Support Vector Machine

In [30]:
support_vector_machine=svm.SVC()

In [32]:
support_vector_machine.fit(tfidf_train, y_train)
pred3 = support_vector_machine.predict(tfidf_test)

print('Accuracy of SVM on test set:',accuracy_score(y_test, pred3))

Accuracy of SVM on test set: 0.9227467811158798


In [33]:
results3=confusion_matrix(y_test, pred3)
print(results3)

[[115  11]
 [  7 100]]


### Pipeline Creation

In [63]:
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf',RandomForestClassifier()),])

In [64]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

In [65]:
pipeline.predict(["All deaths by respiratory failure and pneumonia are being registered as COVID-19, according to the Civil Registry website."])

array(['FAKE'], dtype=object)

In [66]:
#saving the pipeline
filename = 'pipeline.sav'
joblib.dump(pipeline, filename)

['pipeline.sav']

In [67]:
filename = './pipeline.sav'

In [68]:
loaded_model = joblib.load(filename)
result = loaded_model.predict(["One of the challenges being faced worldwide is the rate of infection of health professionals while treating COVID-19 patients.  2016 statistics show the following picture of Rwanda’s health sector: 1/50,505 ground ambulance / population Ratio; 1/10,055 Doctor/population ratio; 1/1,094 Nurse/population ratio; 1/10,500 Laboratory technicians /population ratio. Although Rwanda is on track to meeting most of the required quota, the country cannot afford the set back that a pandemic like COVID-19 among health professionals would cause. To minimize contact time with confirmed cases and therefore reducing the risk of contamination of health professionals in COVID-19 treatment centres, robots were deployed. At Gatenga and Kanyinya treatment centres in Kigali City, visits by medical staff to patients went from 3-4 to 2 per day since May 2020. The 5 human-size robots are programmed to perform temperature screening, take readings of vitals, deliver video messages and detect people not wearing masks then instruct them to wear masks properly. On Tuesday, May 19 at the Kanyinya COVID-19 Treatment Centre, the 5 robots were launched by the Ministry of Health. Named in Kinyarwanda, Akazuba, Ikirezi, Mwiza, Ngabo, and Urumuri are made by Zora Bots, a Belgian company specialised in robotics. They were acquired through a partnership between the UNDP Rwanda Accelerator Lab (AccLab) and the Ministry of ICT and Innovation.  Each robot costs about $30,000. Urumuri is currently deployed at the Kigali International Airport with the capacity to screen 50 to 150 people per minute and report abnormalities to officers on duty. As Rwanda is reopening Kigali International Airport for commercial international flights from 1st of August, Urumuri will allow to speed up mass screenings of fever for passengers as they land. Smart anti-epidemic robots have been used in Asia to help flatten the COVID-19 curve. The pandemic has proven to be a crosscutting challenge affecting Rwanda in all sectors (Health, Economy, Education, Agriculture, …). It therefore requires different sometimes innovative solutions with available resources like robots."])
print(result) 

['TRUE']
